In [0]:
silver_trips_table="databricks_catalog.silver.nyc_taxi_trips"
fact_table="databricks_catalog.gold.fact_trips"

In [0]:
from pyspark.sql.functions import * 
from pyspark.sql.types import * 
from delta.tables import DeltaTable 

####Data Reading

In [0]:
df_silver=spark.table(silver_trips_table)

####Select final fact columns

In [0]:
fact_df = df_silver.select(
    "trip_key",
    col("PULocationID").alias("pickup_location_id"),
    col("DOLocationID").alias("dropoff_location_id"),
    col("VendorID").alias("vendor_id"),
    "payment_type",
    "passenger_count",
    "trip_distance",
    "fare_amount",
    "tip_amount",
    "total_amount",
    col("tpep_pickup_datetime").alias("pickup_datetime"),
    col("tpep_dropoff_datetime").alias("dropoff_datetime"),
    to_date("tpep_pickup_datetime").alias("trip_date")
)

####Write Gold fact (INSERT ONLY)

In [0]:
if not spark.catalog.tableExists(fact_table):
    fact_df.write.format("delta").mode("overwrite").saveAsTable(fact_table)
else:
    dt = DeltaTable.forName(spark, fact_table)
    (
        dt.alias("t")
        .merge(fact_df.alias("s"), "t.trip_key = s.trip_key")
        .whenNotMatchedInsertAll()
        .execute()
    )